# Machine Learning Engineer Nanodegree

## Capstone Project - Code Base

Mathias von Kaiz  
August 1st, 2017

**This code base is to evaluate functions, algorithms for developing the final Robot class.**

In [17]:
import numpy as np

class Robot(object):
    def __init__(self, maze_dim):
        '''
        Use the initialization function to set up attributes that your robot
        will use to learn and navigate the maze. Some initial attributes are
        provided based on common information, including the size of the maze
        the robot is placed in.
        '''

        self.location = [0, 0]
        self.heading = 'up'
        self.maze_dim = maze_dim
        

    def next_move(self, sensors):
        '''
        Use this function to determine the next move the robot should make,
        based on the input from the sensors after its previous move. Sensor
        inputs are a list of three distances from the robot's left, front, and
        right-facing sensors, in that order.

        Outputs should be a tuple of two values. The first value indicates
        robot rotation (if any), as a number: 0 for no rotation, +90 for a
        90-degree rotation clockwise, and -90 for a 90-degree rotation
        counterclockwise. Other values will result in no rotation. The second
        value indicates robot movement, and the robot will attempt to move the
        number of indicated squares: a positive number indicates forwards
        movement, while a negative number indicates backwards movement. The
        robot may move a maximum of three units per turn. Any excess movement
        is ignored.

        If the robot wants to end a run (e.g. during the first training run in
        the maze) then returing the tuple ('Reset', 'Reset') will indicate to
        the tester to end the run and return the robot to the start.
        '''

        rotation = 0
        movement = 20

        print(sensors)
        
        return rotation, movement

In [18]:
from maze import Maze
#from robot import Robot
import sys

# global dictionaries for robot movement and sensing
dir_sensors = {'u': ['l', 'u', 'r'], 'r': ['u', 'r', 'd'],
               'd': ['r', 'd', 'l'], 'l': ['d', 'l', 'u'],
               'up': ['l', 'u', 'r'], 'right': ['u', 'r', 'd'],
               'down': ['r', 'd', 'l'], 'left': ['d', 'l', 'u']}
dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
dir_reverse = {'u': 'd', 'r': 'l', 'd': 'u', 'l': 'r',
               'up': 'd', 'right': 'l', 'down': 'u', 'left': 'r'}

dir_mazes = {'maze1': 'test_maze_01.txt',
            'maze2': 'test_maze_02.txt',
            'maze3': 'test_maze_01.txt'}

# test and score parameters
max_time = 10
train_score_mult = 1/30.

if __name__ == '__main__':
    '''
    This script tests a robot based on the code in robot.py on a maze given
    as an argument when running the script.
    '''

    # Create a maze based on input argument on command line.
    testmaze = Maze( str(dir_mazes['maze1']) )

    # Intitialize a robot; robot receives info about maze dimensions.
    testrobot = Robot(testmaze.dim)

    # Record robot performance over two runs.
    runtimes = []
    total_time = 0
    for run in range(2):
        print "Starting run {}.".format(run)

        # Set the robot in the start position. Note that robot position
        # parameters are independent of the robot itself.
        robot_pos = {'location': [0, 0], 'heading': 'up'}

        run_active = True
        hit_goal = False
        while run_active:
            # check for end of time
            total_time += 1
            if total_time > max_time:
                run_active = False
                print "Allotted time exceeded."
                break

            # provide robot with sensor information, get actions
            sensing = [testmaze.dist_to_wall(robot_pos['location'], heading)
                       for heading in dir_sensors[robot_pos['heading']]]
            rotation, movement = testrobot.next_move(sensing)

            # check for a reset
            if (rotation, movement) == ('Reset', 'Reset'):
                if run == 0 and hit_goal:
                    run_active = False
                    runtimes.append(total_time)
                    print "Ending first run. Starting next run."
                    break
                elif run == 0 and not hit_goal:
                    print "Cannot reset - robot has not hit goal yet."
                    continue
                else:
                    print "Cannot reset on runs after the first."
                    continue

            # perform rotation
            if rotation == -90:
                robot_pos['heading'] = dir_sensors[robot_pos['heading']][0]
            elif rotation == 90:
                robot_pos['heading'] = dir_sensors[robot_pos['heading']][2]
            elif rotation == 0:
                pass
            else:
                print "Invalid rotation value, no rotation performed."

            # perform movement
            if abs(movement) > 3:
                print "Movement limited to three squares in a turn."
            movement = max(min(int(movement), 3), -3) # fix to range [-3, 3]
            while movement:
                if movement > 0:
                    if testmaze.is_permissible(robot_pos['location'], robot_pos['heading']):
                        robot_pos['location'][0] += dir_move[robot_pos['heading']][0]
                        robot_pos['location'][1] += dir_move[robot_pos['heading']][1]
                        movement -= 1
                    else:
                        print "Movement stopped by wall."
                        movement = 0
                else:
                    rev_heading = dir_reverse[robot_pos['heading']]
                    if testmaze.is_permissible(robot_pos['location'], rev_heading):
                        robot_pos['location'][0] += dir_move[rev_heading][0]
                        robot_pos['location'][1] += dir_move[rev_heading][1]
                        movement += 1
                    else:
                        print "Movement stopped by wall."
                        movement = 0

            # check for goal entered
            goal_bounds = [testmaze.dim/2 - 1, testmaze.dim/2]
            if robot_pos['location'][0] in goal_bounds and robot_pos['location'][1] in goal_bounds:
                hit_goal = True
                if run != 0:
                    runtimes.append(total_time - sum(runtimes))
                    run_active = False
                    print "Goal found; run {} completed!".format(run)

    # Report score if robot is successful.
    if len(runtimes) == 2:
        print "Task complete! Score: {:4.3f}".format(runtimes[1] + train_score_mult*runtimes[0])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Starting run 0.
[0, 11, 0]
Movement limited to three squares in a turn.
[0, 8, 0]
Movement limited to three squares in a turn.
[0, 5, 2]
Movement limited to three squares in a turn.
[0, 2, 0]
Movement limited to three squares in a turn.
Movement stopped by wall.
[0, 0, 1]
Movement limited to three squares in a turn.
Movement stopped by wall.
[0, 0, 1]
Movement limited to three squares in a turn.
Movement stopped by wall.
[0, 0, 1]
Movement limited to three squares in a turn.
Movement stopped by wall.
[0, 0, 1]
Movement limited to three s

In [82]:
init = [0, 0]
cost = 1
#delta = [[-1, 0], # go up
#         [ 0,-1], # go left
#         [ 1, 0], # go down
#         [ 0, 1]] # go right

dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
            'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}
delta = ['up', 'right', 'down', 'left']

delta_name = ['^', '<', 'v', '>']

testmaze = Maze( str(dir_mazes['maze1']) )
goal = [testmaze.dim/2 - 1, testmaze.dim/2]

def search(grid, init, goal, cost):
    closed = [[0 for col in range(grid.dim)] for row in range(grid.dim)]
    closed[init[0]][init[1]] = 1
    
    expand = [[-1 for col in range(grid.dim)] for row in range(grid.dim)]
    action = [[-1 for col in range(grid.dim)] for row in range(grid.dim)]

    x = init[0]
    y = init[1]
    g = 0
    
    open = [[g, x, y]]

    found = False
    resign = False
    count = 0

    print ("Start", x, y)
    while not found and not resign:
        if len(open) == 0:
            resign = True
            print ("Failed as not path found")

        else:
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[1]
            y = next[2]
            g = next[0]
            expand[x][y] = count
            count += 1

            # check if we are done
            if x in goal and y in goal:
                found = True
                print ("Goal found", next)

            else:
                for i in range(len(delta)):
                    heading = dir_move[delta[i]]
                    
                    x2 = x
                    y2 = y

                    if testmaze.is_permissible([x2, y2], delta[i]):
                        #print ("Open " + delta[i], testmaze.is_permissible([x2, y2], delta[i]))
                        #print ("Heading " + delta[i])
                        
                        x2 = x + heading[0]
                        y2 = y + heading[1]
                        #print ("New Coords", x2, y2)
                        
                        
                        if closed[x2][y2] == 0:
                            g2 = g + cost
                            open.append([g2, x2, y2])
                            closed[x2][y2] = 1
                            action[x2][y2] = i
                            
                        else:
                            #print ("Heading " + delta[i], "already closed")
                            pass
                    
                    else:
                        #print ("Heading " + delta[i], "movement stopped by wall")
                        pass
                    
    policy = [[' ' for col in range(grid.dim)] for row in range(grid.dim)]
    x = goal[0]
    y = goal[1]
    policy[x][y] = '*'
    while x != init[0] or y != init[1]:
        x2 = x - dir_move[delta[action[x][y]]][0]
        y2 = y - dir_move[delta[action[x][y]]][1]
        policy[x2][y2] = delta_name[action[x][y]]
        x = x2
        y = y2
    
    #return expand
    return policy                       


In [88]:
search(testmaze, init, goal, cost)

('Start', 0, 0)
('Goal found', [30, 6, 6])


array([['^', '<', '<', '<', '^', ' ', ' ', '<', '<', '<', '<', '^'],
       ['^', 'v', ' ', ' ', '^', ' ', '<', 'v', ' ', ' ', ' ', '^'],
       ['<', 'v', ' ', ' ', '<', '<', 'v', ' ', ' ', ' ', ' ', '^'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '^', '>', '>', '>'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '^', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', '^', '>', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', '*', '>', '>', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']], 
      dtype='|S1')